<a href="https://colab.research.google.com/github/jonghhhh/lecture_colabs/blob/main/newspaper3k_032525.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# newspaper3k: 뉴스 기사 본문 스크래핑

## newspaper3k란?
`newspaper3k`는 뉴스 기사 크롤링과 텍스트 추출에 특화된 파이썬 라이브러리입니다. 이 라이브러리를 사용하면 URL만으로 뉴스 기사의 본문, 제목, 작성자, 발행일, 대표 이미지 등을 자동으로 추출할 수 있습니다.

## 주요 기능
- 기사 다운로드 및 파싱
- 기사 제목, 본문, 저자, 발행일 추출
- 메타데이터(og 태그 등) 추출
- 대표 이미지 URL 추출
- 기사 요약 및 키워드 추출(영어 기사만 지원)
- 언어 감지 및 다국어 지원

## 한계점
- 네이버 뉴스와 같은 특정 플랫폼에서는 제한적으로 작동
- 일부 방송사 뉴스 웹사이트에서 완벽하게 작동하지 않을 수 있음
- NLP 기반 요약 및 키워드 추출은 영어 기사에 최적화됨

## newspaper3k 주요 속성 설명

`newspaper3k`로 추출할 수 있는 기사의 주요 속성들은 다음과 같습니다:

| 속성 | 설명 |
|------|------|
| `article.title` | 기사의 제목 |
| `article.authors` | 기사의 저자 리스트(meta_data에서 추출) |
| `article.publish_date` | 발행일(날짜) |
| `article.text` | 기사의 본문 텍스트 |
| `article.top_image` | 기사에서 가장 대표적인 이미지의 URL |
| `article.movies` | 기사 내 포함된 동영상 URL 리스트(영어 기사에 최적화) |
| `article.summary` | NLP 처리 후 생성된 기사의 요약문(영어 기사에만 적용) |
| `article.keywords` | NLP 처리 후 추출된 키워드 리스트(영어 기사에만 적용) |
| `article.meta_data` | HTML 메타데이터 (예: og 태그, 트위터 카드 등) |
| `article.meta_description` | HTML 헤드 영역에 정의된 설명 |
| `article.images` | 기사에 포함된 모든 이미지 URL 리스트 |
| `article.canonical_link` | 기사의 표준 URL |

이 중 일부 속성은 `article.parse()` 메서드만 실행하면 추출되고, 요약문과 키워드와 같은 일부 속성은 `article.nlp()` 메서드를 실행해야 추출됩니다.

In [ ]:
# 필요한 라이브러리 설치
!pip install newspaper3k "lxml[html_clean]" tqdm

In [ ]:
# 필요한 라이브러리 임포트
from newspaper import Article, Config
import json
import time
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Image

## 기본 설정

In [ ]:
# newspaper 설정
config = Config()
config.browser_user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
config.request_timeout = 10
config.memoize_articles = False  # 캐싱 비활성화

# NLP 처리를 위한 NLTK 데이터 다운로드 (최초 1회만 실행, 영어 기사에만 적용됨)
# 한국어 기사만 분석할 예정이라면 아래 코드는 실행할 필요 없음
# import nltk
# nltk.download('punkt')

## 뉴스 기사 스크래핑 및 파싱 함수

In [ ]:
def extract_article_data(url, config=None):
    """
    URL에서 기사 정보를 추출하는 함수

    Args:
        url (str): 뉴스 기사 URL
        config (newspaper.Config, optional): newspaper3k 설정

    Returns:
        dict: 추출된 기사 정보를 담은 딕셔너리
    """
    try:
        # Config 객체가 전달되면 해당 설정으로 Article 객체 생성
        if config:
            article = Article(url, language='ko', config=config)
        else:
            article = Article(url, language='ko')

        # 다운로드 및 파싱
        article.download()
        time.sleep(2)  # 다운로드 후 잠시 대기 (서버 부하 고려)
        article.parse()

        # NLP 처리 (영어 텍스트에 최적화되어 있음, 한국어에서는 제한적 기능)
        # article.nlp()

        # 제목: article.title 우선, 없으면 meta_data의 og:title 사용
        title = article.title if article.title else article.meta_data.get('og', {}).get('title')

        # 발행일: article.publish_date 또는 meta_data의 article 내 published_time
        date_raw = article.publish_date if article.publish_date else article.meta_data.get('article', {}).get('published_time')
        date = str(date_raw).split('+')[0].replace('T', '-').replace(':', '-') if date_raw else None

        # 본문
        text = article.text

        # 사이트: meta_data의 og:site_name (대부분 언론사의 이름)
        site = article.meta_data.get('og', {}).get('site_name')

        # 저자: article.authors가 있으면 문자열로 합치고, 없으면 meta_data의 dable 내 author 사용
        if article.authors and len(article.authors) > 0:
            author = ", ".join(article.authors)
        else:
            author = article.meta_data.get('dable', {}).get('author')

        # 메타 설명: article.meta_description 우선, 없으면 og:description
        description = article.meta_description if article.meta_description else article.meta_data.get('og', {}).get('description')

        # 키워드: meta_data의 news_keywords 우선, 없으면 og:keywords
        keywords = article.meta_data.get('news_keywords') or article.meta_data.get('og', {}).get('keywords')

        # 섹션: meta_data의 article 내 section
        section = article.meta_data.get('article', {}).get('section')

        # 기사 유형: og:type
        type = article.meta_data.get('og', {}).get('type')

        # 대표 이미지: article.top_image 우선, 없으면 og:image
        top_image = article.top_image if article.top_image else article.meta_data.get('og', {}).get('image')

        # 추가: 모든 이미지 URL 목록
        images = list(article.images)

        # 추가: 기사 URL (원본 또는 표준 링크)
        article_url = article.canonical_link or url

        # 결과 딕셔너리
        result = {
            "url": article_url,
            "site": site,
            "date": date,
            "title": title,
            "text": text,
            "author": author,
            "description": description,
            "keywords": keywords,
            "section": section,
            "type": type,
            "top_image": top_image,
            "images": images[:5] if images else []  # 최대 5개 이미지만 저장
        }

        return result, None

    except Exception as e:
        return None, f"Error processing {url}: {str(e)}"

## 뉴스 기사 추출 실행

In [ ]:
# 분석할 기사 URL 리스트
urls = [
    "https://www.domin.co.kr/news/articleView.html?idxno=1500714",  # 지역지
    "https://www.ohmynews.com/NWS_Web/View/at_pg.aspx?CNTN_CD=A0003099071&CMPT_CD=P0010&utm_source=naver&utm_medium=newsearch&utm_campaign=naver_news",  # 인터넷언론
    "https://www.sisajournal-e.com/news/articleView.html?idxno=218656",  # 잡지
    "https://news.tvchosun.com/site/data/html_dir/2025/02/03/2025020390247.html?_gl=1*60n6cu*_ga*MTY2MDQ5OTkwNy4xNzM4NTgxMTYw*_ga_D5GZR50LJV*MTczODYzODM5MS4yLjEuMTczODYzODQxOC4zMy4wLjA.",  # 방송1
    "https://news.sbs.co.kr/news/endPage.do?news_id=N1007969124&plink=THUMB&cooper=SBSNEWSPROGRAM",  # 방송2
    "https://www.yna.co.kr/view/AKR20250130040200053?input=1195m",  # 통신사
    "https://m.skyedaily.com/news_view.html?ID=256827"  # 추가 URL
]
# 결과를 저장할 리스트
results = []
errors = []

# 진행 상황 표시줄과 함께 URL 처리
for url in tqdm(urls, desc="Processing articles"):
    result, error = extract_article_data(url, config)
    if result:
        results.append(result)
    if error:
        errors.append(error)
        print(error)  # 오류 발생 시 바로 출력

# 처리 결과 통계
print(f"\n처리 결과: 성공 {len(results)}개, 실패 {len(errors)}개")

# 최종 결과를 JSON 파일로 저장
with open("news_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

# 최종 결과를 dataframe로 저장
pd.DataFrame(results).to_excel("news_results.xlsx", index=False)

# newspaper3k와 다른 스크래핑 방법 비교

### 1. newspaper3k vs BeautifulSoup

| 특성 | newspaper3k | BeautifulSoup |
|------|-------------|---------------|
| **목적** | 뉴스 기사 전용 스크래핑 | 일반적인 웹 스크래핑 |
| **장점** | - 기사 본문 자동 추출<br>- 메타데이터 자동 추출<br>- 언어 감지 및 다국어 지원<br>- 간단한 API | - 유연한 HTML 파싱<br>- 정교한 선택자 지원<br>- 맞춤형 추출 가능 |
| **단점** | - 일부 사이트에서 제한적<br>- NLP는 영어 중심<br>- 사이트별 맞춤화 어려움 | - 코드가 상대적으로 복잡<br>- 기사 추출 로직 직접 구현 필요<br>- 사이트 구조 변경 시 유지보수 필요 |
| **적합한 경우** | - 다수의 일반적인 뉴스 사이트 분석<br>- 빠른 프로토타이핑<br>- 간단한 뉴스 수집 | - 특정 사이트 맞춤형 스크래핑<br>- 복잡한 웹페이지 구조 처리<br>- newspaper3k가 작동하지 않는 사이트 |

### 2. newspaper3k vs Trafilatura

[Trafilatura](https://github.com/adbar/trafilatura)는 웹 콘텐츠에서 텍스트를 추출하는 또 다른 라이브러리입니다.

| 특성 | newspaper3k | Trafilatura |
|------|-------------|-------------|
| **목적** | 뉴스 기사 전용 스크래핑 | 다양한 웹 콘텐츠 추출 |
| **장점** | - 뉴스 메타데이터 추출<br>- NLP 기능 내장<br>- 간단한 API | - 더 정확한 본문 추출<br>- 블로그, 포럼 등 다양한 콘텐츠 지원<br>- 구조화된 출력(XML 등) |
| **처리 속도** | 보통 | 빠름 |
| **적합한 경우** | - 메타데이터가 중요한 경우<br>- 뉴스 기사 전용 분석 | - 본문 텍스트 정확도가 중요한 경우<br>- 다양한 형식의 웹 콘텐츠 처리 |

### 3. urllib/requests 직접 사용 비교

urllib/requests를 직접 사용하여 스크래핑하는 방법과 newspaper3k의 주요 차이점:

| 특성 | newspaper3k | urllib/requests+BeautifulSoup |
|------|-------------|----------------------------|
| **코드 간결성** | 매우 간결 (3-4줄) | 복잡 (30-50줄 이상) |
| **유지보수** | 간편 (라이브러리가 알고리즘 업데이트) | 사이트 변경 시 재작성 필요 |
| **추출 정확도** | 보통 (사이트별로 차이) | 맞춤화 시 높음 |
| **개발 시간** | 짧음 | 길음 |
| **맞춤화** | 제한적 | 완전한 맞춤화 가능 |

### 사용 권장 시나리오

1. **newspaper3k 권장**:
   - 다수의 일반적인 뉴스 사이트에서 빠르게 데이터 수집
   - 프로토타이핑 및 탐색적 분석
   - 개발 시간이 제한적인 프로젝트

2. **BeautifulSoup/Trafilatura 권장**:
   - newspaper3k가 제대로 작동하지 않는 사이트
   - 특정 사이트에 맞춤화된 정밀한 추출 필요
   - 뉴스 이외의 콘텐츠(블로그, 포럼 등) 추출